In [138]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/tourism-data/test/test.xlsx
/kaggle/input/tourism-data/test/result1.csv
/kaggle/input/tourism-data/train/2018-2019.xlsx
/kaggle/input/tourism-data/train/2020-2021.xlsx


In [139]:
from matplotlib import pyplot as plt
import seaborn as sns
import warnings
import jieba
import re
%matplotlib inline
warnings.filterwarnings('ignore')

# Task 1

In [140]:
data1 = pd.read_excel('/kaggle/input/tourism-data/train/2018-2019.xlsx', sheet_name=4)
data2 = pd.read_excel('/kaggle/input/tourism-data/train/2020-2021.xlsx', sheet_name=4)
test_data = pd.read_excel('../input/tourism-data/test/test.xlsx')

In [141]:
data1.head()

,文章ID,公众号标题,发布时间,正文
0,1001,2018，对自己好一点,2018-01-02 17:28,2017的旅程已经结束\n2018的未来拉开了帷幕\n新的一年里，请对自己好一点\n一辈子很...
1,1002,春节机票预订有窍门,2018-01-02 17:28,距离春节还有一个多月的时间，在线旅游网站的春节机票销售火爆，部分航线甚至一票难求。在这里分享...
2,1003,冬日旅游知多D,2018-01-03 17:32,960万平方公里的祖国大地，四季都有独特美景\n冬天的旅行也别有一番风味\n但是冬季的严寒气...
3,1004,2018冬季暖心之旅,2018-01-03 17:32,长按二维码，关注我们\n中心联系人：林小姐13709649096\n刘小姐135000781...
4,1005,关于粤K27618号大客车排气管“喷火”事件的情况说明,2018-01-05 16:57,


In [142]:
data2.head()

,文章ID,公众号标题,发布时间,正文
0,1622,元旦晚8点，茂名海鲜亮相央视！一定要看！,2020-01-01 19:22,新年伊始，你的打开方式是怎样的呢？是准备去享受一顿丰盛的大餐？还是将“世界那么大，我想去看看...
1,1623,建好省级全域旅游示范区，打造沿海经济带新增长极 茂名市电白区旅游协会举行第二届第二次全体会议,2020-01-05 02:10,会员报到\n\n会场\n\n 为了更好地贯彻落实区委区政府和区文广旅体局建设全...
2,1624,【茂名交投旅游】@茂名人，信宜旅游景点上新啦~,2020-01-06 17:27,好消息！好消息！\n\n\n信宜莲花湖庄园\n\n开业啦~~~\n庄园简介\n\n\n\n\...
3,1625,【茂名交投旅游】2020，打卡鼎龙湾新玩法！,2020-01-06 17:27,2020.首波惊喜狂潮\n鼎龙湾·德萨斯牛仔小镇全面升级\n1月11日\n牛仔主题嘉年华街区...
4,1626,茂名 | 串珠成链，建滨海旅游目的地,2020-01-08 21:35,地处粤港澳大湾区、北部湾城市群、海南自贸区三大国家战略交汇处的茂名，紧紧抓住沿海经济带与大湾...


In [143]:
columns = ['ID', 'Title', 'Date', 'Content']
data1.columns = columns
data2.columns = columns

In [144]:
data = pd.concat([data1, data2], ignore_index=True)
data['Content'] = data['Title'] + '\n' + data['Content']
data.drop(columns=['Title', 'Date'], inplace=True)
data.head()

,ID,Content
0,1001,2018，对自己好一点\n2017的旅程已经结束\n2018的未来拉开了帷幕\n新的一年里，...
1,1002,春节机票预订有窍门\n距离春节还有一个多月的时间，在线旅游网站的春节机票销售火爆，部分航线甚...
2,1003,冬日旅游知多D\n960万平方公里的祖国大地，四季都有独特美景\n冬天的旅行也别有一番风味\...
3,1004,2018冬季暖心之旅\n长按二维码，关注我们\n中心联系人：林小姐13709649096\n...
4,1005,关于粤K27618号大客车排气管“喷火”事件的情况说明\n ...


In [145]:
data.dropna(inplace=True)

In [146]:
test_data.head()

,文章ID,公众号标题,正文
0,8396,@茂名家长，广东全面启动3-11岁儿童新冠疫苗接种！是否安全？最全问答来了→,按照国家整体部署\n广东省全面启动\n3-11岁人群新冠病毒疫苗接种\n3-11岁人群接种哪...
1,8397,庄悦群：把安全生产摆在首要位置抓紧抓实，加快推进绿色矿山智慧矿山建设,10月31日上午，市委副书记、市长庄悦群带队到高州市，就深入贯彻落实习近平总书记关于安全生产...
2,8398,群众办事百项堵点疏解行动,为破解群众办事堵点难点，发展改革委会同新华社、中国政府网于4月19日启动了“群众办事百项...
3,8399,茂名启动台风Ⅲ级应急响应！市旅游局发布防御台风安全提示,点击上方蓝字，关注我们\n\n\n台风最新消息\n \n目前，台风正朝着广东袭来\n↓↓↓\...
4,8400,灾情就是命令,受南海热带低压影响，我市近日受强降雨影响，钱排、合水、新宝、平塘、思贺等镇遭受特大暴雨袭击，...


In [147]:
test_data.columns = ['ID', 'Title', 'Content']
test_data['Content'] = test_data['Title'] + '\n' + test_data['Content']
test_data.drop(columns=['Title'], inplace=True)
test_data.head()

,ID,Content
0,8396,@茂名家长，广东全面启动3-11岁儿童新冠疫苗接种！是否安全？最全问答来了→\n按照国家整体...
1,8397,庄悦群：把安全生产摆在首要位置抓紧抓实，加快推进绿色矿山智慧矿山建设\n10月31日上午，市...
2,8398,群众办事百项堵点疏解行动\n 为破解群众办事堵点难点，发展改革委会同新华社、中国政府网于4...
3,8399,茂名启动台风Ⅲ级应急响应！市旅游局发布防御台风安全提示\n点击上方蓝字，关注我们\n\n\n...
4,8400,灾情就是命令\n受南海热带低压影响，我市近日受强降雨影响，钱排、合水、新宝、平塘、思贺等镇遭...


In [148]:
print('Train data shape:', data.shape)
print('Test data shape:', test_data.shape)
train_size = data.shape[0]

Train data shape: (6285, 2)
Test data shape: (930, 2)


In [149]:
data = pd.concat([data, test_data], ignore_index=True)
data.shape

(7215, 2)

In [150]:
def preprocessing(data):
    try:
        lines = data.split()
    except Exception as e:
        print(data)
        return ''
    lines = list(filter(lambda x: x is not '', lines))
    unuse_lis = []
    rule_1 = r'\W'
    compiled_rule_1 = re.compile(rule_1)
    for line in lines:
        no_en_and_da = compiled_rule_1.findall(line)
        no_en_and_da_str = ''.join(no_en_and_da)
        reslis = re.findall(r'^\S', ''.join(re.findall(r'[^\，]', ''.join(re.findall(r'[^\。]', no_en_and_da_str)))))
        unuse_lis.append(reslis)
    syms = []
    for i in unuse_lis:
        for j in i:
            syms.append(j)
    syms = list(set(syms))
    
    def replace_syms(line):
        for sym in syms:
            line = line.replace(sym, '')
        return line
    
    def replace_lem(line):
        a = re.sub(r'\s', '', line)
        b = re.sub(r'\W{2,}', '', a)
        c = re.sub(r'\d', '', b)
        d = re.sub(r' ', '', c)
        d = d.replace('_', '')
        return d
    
    lines = list(map(replace_syms, lines))
    lines = list(map(replace_lem, lines))
    lines = list(filter(lambda x: x not in '1234567890', lines))
    
    res = []
    for line in lines:
        for word in jieba.cut(line):
            if word == '，':
                continue
            res.append(word)
    return ' '.join(res)

In [151]:
preprocessing(data.loc[0, 'Content'])

'对 自己 好 一点 的 旅程 已经 结束 的 未来 拉开 了 帷幕 新 的 一年 里 请 对 自己 好 一点 一辈子 很长 漫漫 人生 路上 会 遇到 那么 多 的 人和事 一辈子 很 短 一年 一年 时光 悄悄 从 指缝间 溜走 在 有限 的 生命 里 从 现在 开始 享受 自己 的 人生 不 强求 缘分 该 遇见 的 人 总会 遇见 留不住 的 过客 终将 远走 有缘 相守 心怀 感恩 无缘 结交 坦然 放手 不 强求 你 我 都 是 自由 的 随缘 就 好 陪伴 家人 站 在 我们 身后 的 家人 是 前进 时 的 坚强 后盾 给予 我们 无条件 的 支持 是 孤独 无助 时 的 避风港 默默 付出 而 不求 回报 把 时间 留 一点 给 家人 他们 所 需 的 不过 是 陪伴 珍惜 朋友 时光流逝 仿佛 大浪淘沙 留下来 的 真心 朋友 不多不少 刚刚 好 三五 知己 小酌 几杯 君子之交 虽淡 但 贵在 真心 足矣 认真 工作 新 的 一年 里 全身心 投入 工作 付出 会 有 回报 工作 会为 你 搭 起 梦想 与 人生价值 之间 的 桥梁 认真 工作 别 辜负 自己 爱护 身体 健康 的 体魄 是 自己 最大 的 本钱 闲暇 时 散散步 锻炼 一下 身体 用 自律 而 健康 的 作息 对 自己 的 身体 负责 出去 走走 心灵 也 需要 放松 与 远行 城市 里 的 拥挤 高楼 令人 迷茫 不妨 出去 走走 行万里路 感受 万里 风光 体验 形色 人生 过去 已经 过去 未来 即将 到来 新 的 一年 愿 你 想要 的 明天 都 终将 到来 更 多 精彩 线路 欢迎 关注 茂名 交投 旅游 集散中心 站 前 四路 交委 车站 一楼 中心 联系人 林小姐 刘小姐 罗 小姐 陈小姐 市区 联系人 柯 小姐 柯 先生 吴小姐 梁小姐 邓 先生 电白 分部 蔡 小姐 高州 分部 唐小姐 化州 分部 马先生 李小姐 信宜 分部 陈小姐 谢小姐'

In [152]:
data['Cut'] = data['Content'].apply(preprocessing)
data.head()

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan


,ID,Content,Cut
0,1001,2018，对自己好一点\n2017的旅程已经结束\n2018的未来拉开了帷幕\n新的一年里，...,对 自己 好 一点 的 旅程 已经 结束 的 未来 拉开 了 帷幕 新 的 一年 里 请 对...
1,1002,春节机票预订有窍门\n距离春节还有一个多月的时间，在线旅游网站的春节机票销售火爆，部分航线甚...,春节 机票 预订 有 窍门 距离 春节 还有 一个多月 的 时间 在线 旅游 网站 的 春节...
2,1003,冬日旅游知多D\n960万平方公里的祖国大地，四季都有独特美景\n冬天的旅行也别有一番风味\...,冬日 旅游 知多 D 万平方公里 的 祖国 大地 四季 都 有 独特 美景 冬天 的 旅行 ...
3,1004,2018冬季暖心之旅\n长按二维码，关注我们\n中心联系人：林小姐13709649096\n...,冬季 暖心 之 旅 长 按 二维码 关注 我们 中心 联系人 林小姐 刘小姐 罗 小姐 陈小...
4,1005,关于粤K27618号大客车排气管“喷火”事件的情况说明\n ...,关于 粤 K 号 大客车 排气管 喷火 ” 事件 的 情况 说明


In [153]:
data.shape

(7215, 3)

## 1.1 By LDA

In [154]:
data['AsList'] = data['Cut'].apply(lambda x: list(filter(lambda x: ' ' not in x, jieba.lcut(x))))
data['AsList'].head()

KeyboardInterrupt: 

In [ ]:
text_data = data['AsList'].tolist()

In [ ]:
from gensim import corpora
from gensim.models import TfidfModel

dictionary = corpora.Dictionary(text_data)
dictionary.filter_n_most_frequent(200)
corpus = [dictionary.doc2bow(text) for text in text_data]

# tfidf = TfidfModel(corpus)
# tfidf.save('task_1_tfidf.model')
# # corpus = tfidf[corpus]

In [ ]:
dictionary.save('task_1.dict')

In [ ]:
from gensim.models import LdaModel

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 200
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)

model.save('task_1.model')  # 将模型保存到硬盘

In [ ]:
topic_list = model.print_topics()
topic_list = sorted(topic_list, key=lambda x: x[0])
print(topic_list)

In [ ]:
top_topics = model.top_topics(corpus)

# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
print('Average topic coherence: %.4f.' % avg_topic_coherence)

# from pprint import pprint
# pprint(top_topics)

In [ ]:
lda_topics = []
for topic in top_topics:
    topic_list, _ = topic
    lda_topics.append([x[1] for x in topic_list])
lda_topics

In [ ]:
test_doc = text_data[23]
print(test_doc)
doc_bow = dictionary.doc2bow(test_doc)
# doc_tfidf = tfidf[doc_bow]
doc_lda = model[doc_bow]
doc_lda

In [ ]:
def judge_coherence(doc):
    coh_dict = ['旅游', '活动', '节庆', '特产', '交通', '酒店', '景区', '景点',
                '文创', '文化', '乡村旅游', '民宿', '假日', '假期', '游客', '采摘',
                '赏花', '春游', '踏青', '康养', '公园', '滨海游', '度假', '农家乐',
                '剧本杀', '旅行', '徒步', '工业旅游', '线路', '自驾游', '团队游',
                '攻略', '游记', '包车', '玻璃栈道', '游艇', '高尔夫', '温泉']
    doc_bow = dictionary.doc2bow(test_doc)
#     doc_tfidf = tfidf[doc_bow]
    doc_lda = model[doc_bow]
    topic_idx = [each[0] for each in doc_lda]
    topics = []
    for idx in topic_idx:
        topics.extend(lda_topics[idx - 1])
    
    def judge(topics, coh_dict):
        for x in topics:
            for y in coh_dict:
                if y in x:
                    return True
        return False
    
    return judge(topics, coh_dict)

In [ ]:
test_text = text_data[train_size:]
test_res = [judge_coherence(text) for text in test_text]

## 1.2 By TF-IDF and TextRank

In [159]:
import jieba.analyse

def judge_by_tfidf(text):
    coh_dict = ['旅游', '活动', '节庆', '特产', '交通', '酒店', '景区', '景点',
                '文创', '文化', '乡村旅游', '民宿', '假日', '假期', '游客', '采摘',
                '赏花', '春游', '踏青', '康养', '公园', '滨海游', '度假', '农家乐',
                '剧本杀', '旅行', '徒步', '工业旅游', '线路', '自驾游', '团队游',
                '攻略', '游记', '包车', '玻璃栈道', '游艇', '高尔夫', '温泉']
    keywords = jieba.analyse.extract_tags(text, topK=50, withWeight=False)
    for x in keywords:
        for y in coh_dict:
            if y in x:
                return True
    return False

def judge_by_textrank(text):
    coh_dict = ['旅游', '活动', '节庆', '特产', '交通', '酒店', '景区', '景点',
                '文创', '文化', '乡村旅游', '民宿', '假日', '假期', '游客', '采摘',
                '赏花', '春游', '踏青', '康养', '公园', '滨海游', '度假', '农家乐',
                '剧本杀', '旅行', '徒步', '工业旅游', '线路', '自驾游', '团队游',
                '攻略', '游记', '包车', '玻璃栈道', '游艇', '高尔夫', '温泉']
    keywords = jieba.analyse.textrank(text, topK=50, withWeight=False)
    for x in keywords:
        for y in coh_dict:
            if y in x:
                return True
    return False

def judge(text):
    return '相关' if judge_by_tfidf(text) and judge_by_textrank(text) else '不相关'

test_res = [judge(text) for text in data[train_size:]['Cut'].tolist()]

In [158]:
task1_res = pd.DataFrame({'文章ID': test_data['ID'].tolist(), 
                          '分类标签': test_res})
task1_res.to_csv('result1.csv', index=False)

# Task 2

In [ ]:
def read_in(path):
    sheet0 = pd.read_excel(path, sheet_name=0)
    sheet1 = pd.read_excel(path, sheet_name=1)
    sheet2 = pd.read_excel(path, sheet_name=2)
    sheet3 = pd.read_excel(path, sheet_name=3)
    sheet0.columns = ['ID', 'City', 'Name', 'RevDate', 'Content', 'CheckDate', 'HouseType']
    sheet0['ID'] = sheet0['ID'].apply(lambda x: '酒店评论-' + str(x))
    sheet0 = sheet0[['ID', 'Content']]
    sheet1.columns = ['ID', 'City', 'Name', 'Date', 'Content']
    sheet1['ID'] = sheet1['ID'].apply(lambda x: '景区评论-' + str(x))
    indices = sheet1[sheet1['Name'].str.contains('湛江|广东海洋大学|南极长城站')].index
    sheet1.drop(index=indices, inplace=True)
    sheet1 = sheet1[['ID', 'Content']]
    sheet2 = sheet2[['游记ID', '正文']]
    sheet2.columns = ['ID', 'Content']
    sheet2['ID'] = sheet2['ID'].apply(lambda x: '游记-' + str(x))
    sheet3 = sheet3[['餐饮评论ID', '评论内容', '标题']]
    sheet3.columns = ['ID', 'Content', 'Title']
    sheet3['Content'] = sheet3['Title'] + '\n' + sheet3['Content']
    sheet3.drop(columns='Title', inplace=True)
    sheet3['ID'] = sheet3['ID'].apply(lambda x: '餐饮评论-' + str(x))
    ret = pd.concat([sheet0, sheet1, sheet2, sheet3], ignore_index=True)
    return ret

In [ ]:
data1 = read_in('../input/tourism-data/train/2018-2019.xlsx')
data2 = read_in('../input/tourism-data/train/2020-2021.xlsx')
data = pd.concat([data1, data2], ignore_index=True)
data.head(), data.shape

In [ ]:
data['Content'] = data['Content'].apply(lambda x: re.sub(r'^.*?\n\d+\-\d+\-\d+.*?\nhttp.*?\n|\n.*?\d+\-\d+\-\d+.*?\nhttp.*?\n|\d+\-\d+\-\d+.*?\nhttp.*?\n| ', '', x))
data['Content'].dropna()
data.head(), data.shape

In [169]:
!pip install pyhanlp, foolnltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.0/137.0 kB 858.5 kB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 9.5 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for pyhanlp: filename=pyhanlp-0.1.84-py3-none-any.whl size=29818 sha256=5443f4224bc371b6c1ec8c295b219410ecbd6796efa4522019ece0c89788821d
  Stored in directory: /root/.cache/pip/wheels/8e/67/74/f1dfe4d6ea994941a47aee14de63ca71aba2ab96e2e217c072
  Created wheel for hanlp-downloader: filename=hanlp_downloader-0.0.25-py3-none-any.whl size=13776 sha256=8cbd2e70d4e59d33e142d4df625269c9e919a8b15d98fba13d3dce5119e3576f
  Stored in directory: /root/.cache/pip/wheels/cb/a2/c3/61f7b80f5d56e5b19dd9cb50303f8f3b621662f035a281d197
Successfully built pyhanlp hanlp-downloader


In [ ]:
from pyhanlp import *
import fool

data['Content'] = data['Content'].apply(HanLP.convertToSimplifiedChinese)